In [1]:
import time
notebook_start_time = time.perf_counter()

# NetOGlyc4 data high quality validation/test set partitioning

## Imports

### Built-in imports

In [2]:
import math
import gzip
import pickle
from pathlib import Path
import warnings
import re
from time import sleep
import os
from urllib.parse import urlparse, parse_qs
from decimal import Decimal

### Shared library imports

In [3]:
import glyc_processing.annotation

### External imports

In [4]:
import numpy as np
import pandas as pd
import h5py
from tqdm.auto import tqdm
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Markdown

## Paths & Constants

In [5]:
#BASE_DIR = Path("/mnt/g/My Drive/CloudVault/Masters/Data")
BASE_DIR = Path("/home/jakob/Cloudvault_new/Data")

# Path of exported annotations file
ANNOTATIONS_FILE = BASE_DIR/'NetOGlyc5 data'/'GalNAc data'/'04-annotation'/'netoglyc4_protein_annotations.pkl.gz'

# Path of directory to put Graphpart input/output (will be created)
GRAPHPART_DIR = BASE_DIR/'NetOGlyc5 data'/'GalNAc data'/'06-partitioning'/'netoglyc4_graphpart_partitions'

# Maximum sequence length of proteins for valid/test sets
MAX_SEQ_LENGTH = 10000

# Homology threshold that graph-part should use
GRAPHPART_THRESHOLD = 0.3

In [6]:
GRAPHPART_INPUT_FILE = GRAPHPART_DIR/'graphpart_input.fasta'
GRAPHPART_OUTPUT_FILE = GRAPHPART_DIR/'graphpart_output.csv'

## Fetch protein annotations

In [7]:
with gzip.open(ANNOTATIONS_FILE, 'rb') as f:
    proteins = pickle.load(f)

In [8]:
proteins.scoring_function = glyc_processing.annotation.max_score

## Split proteins by presence of nonambiguous/ambiguous/non-glcyosylated sites

In [9]:
trunc_n_sites = 0
trunc_non_glyc_proteins = {}
trunc_glyc_proteins_nonglyc = {}
trunc_glyc_proteins = {}
trunc_certain_glyc_proteins = {}

for protein in proteins.values():
    protein_id = protein.protein_id
    protein_len = min(len(protein.protein_seq), MAX_SEQ_LENGTH)

    glycosylation_labels = np.array(protein.get_glycosylation_labels(end=protein_len))

    sites_mask = glycosylation_labels >= 0
    trunc_n_sites += sites_mask.sum()

    glyc_sites_mask = glycosylation_labels > 0
    if glyc_sites_mask.sum() > 0:
        trunc_glyc_proteins[protein_id] = np.where(glyc_sites_mask)[0]

    non_glyc_sites_mask = glycosylation_labels == 0
    if glyc_sites_mask.sum() == 0:
        trunc_non_glyc_proteins[protein_id] = np.where(non_glyc_sites_mask)[0]
    else:
        trunc_glyc_proteins_nonglyc[protein_id] = np.where(non_glyc_sites_mask)[0]

    certain_glyc_sites_mask = glycosylation_labels == 1
    if certain_glyc_sites_mask.sum() > 0:
        trunc_certain_glyc_proteins[protein_id] = np.where(certain_glyc_sites_mask)[0]

print(f"Proteins: {len(proteins)}")
print(f"\tNon-glycosylated Proteins: {len(trunc_non_glyc_proteins)} ({len(trunc_non_glyc_proteins)/len(proteins)*100:.2f}%)")
print(f"\tGlycosylated Proteins: {len(trunc_glyc_proteins)} ({len(trunc_glyc_proteins)/len(proteins)*100:.2f}%)")
print(f"\t\tGlycosylated Proteins w. (at least 1) Un-ambiguous site: {len(trunc_certain_glyc_proteins)} ({len(trunc_certain_glyc_proteins)/len(proteins)*100:.2f}%)")
print()

trunc_non_glyc_proteins_non_glyc_sites = sum(len(trunc_non_glyc_proteins[protein_id]) for protein_id in trunc_non_glyc_proteins)
trunc_glyc_proteins_non_glyc_sites = sum(len(trunc_glyc_proteins_nonglyc[protein_id]) for protein_id in trunc_glyc_proteins_nonglyc)
trunc_glyc_sites = sum(len(trunc_glyc_proteins[protein_id]) for protein_id in trunc_glyc_proteins)
trunc_certain_glyc_sites = sum(len(trunc_certain_glyc_proteins[protein_id]) for protein_id in trunc_certain_glyc_proteins)

print(f"Sites: {trunc_n_sites}")
print(f"\tNon-glycosylated Sites: {trunc_glyc_proteins_non_glyc_sites+trunc_non_glyc_proteins_non_glyc_sites} ({(trunc_glyc_proteins_non_glyc_sites+trunc_non_glyc_proteins_non_glyc_sites)/trunc_n_sites*100:.2f}%)")
print(f"\t\tNon-glycosylated Sites in Glycosylated Proteins: {trunc_glyc_proteins_non_glyc_sites} ({trunc_glyc_proteins_non_glyc_sites/trunc_n_sites*100:.2f}%)")
print(f"\t\tNon-glycosylated Sites in Non-glycosylated Proteins: {trunc_non_glyc_proteins_non_glyc_sites} ({trunc_non_glyc_proteins_non_glyc_sites/trunc_n_sites*100:.2f}%)")
print(f"\tGlycosylated Sites: {trunc_glyc_sites} ({trunc_glyc_sites/trunc_n_sites*100:.2f}%)")
print(f"\t\tNon-ambiguous Glycosylated Sites: {trunc_certain_glyc_sites} ({trunc_certain_glyc_sites/trunc_n_sites*100:.2f}%)")

Proteins: 786
	Non-glycosylated Proteins: 310 (39.44%)
	Glycosylated Proteins: 476 (60.56%)
		Glycosylated Proteins w. (at least 1) Un-ambiguous site: 476 (60.56%)

Sites: 5329
	Non-glycosylated Sites: 3255 (61.08%)
		Non-glycosylated Sites in Glycosylated Proteins: 872 (16.36%)
		Non-glycosylated Sites in Non-glycosylated Proteins: 2383 (44.72%)
	Glycosylated Sites: 2074 (38.92%)
		Non-ambiguous Glycosylated Sites: 2074 (38.92%)


In [23]:
glyc_sites_count = 0
nonglyc_sites_count = 0
nonglyc_residues_count = 0

for protein in proteins.values():
    for idx, count in enumerate(protein.seq_idx_seen_count[:1022]):
        if protein.seq_sites[idx] is not None:
            if len(protein.seq_sites[idx]) > 0:
                glyc_sites_count += 1
            else:
                nonglyc_sites_count += 1
        else:
            nonglyc_residues_count += 1

glycosylatable_sites_count = glyc_sites_count+nonglyc_sites_count
residues_count = glyc_sites_count+nonglyc_residues_count

glycosylatable_glyc_weight = glycosylatable_sites_count/(2*glyc_sites_count)
glycosylatable_nonglyc_weight = glycosylatable_sites_count/(2*nonglyc_sites_count)

all_glyc_weight = residues_count/(2*glyc_sites_count)
all_nonglyc_weight = residues_count/(2*nonglyc_residues_count)

glycosylatable_pos_weight = glycosylatable_glyc_weight / glycosylatable_nonglyc_weight
all_pos_weight = all_glyc_weight / all_nonglyc_weight

print(f"Total residues:{residues_count}")  
print(f"Total sites:{glycosylatable_sites_count}")  
print(f"Glycosylated sites:\t{glyc_sites_count}\t{glycosylatable_sites_count/glycosylatable_sites_count*100:.2f}%")

print(f"Seen non-glycosylated sites:\t{nonglyc_sites_count}\t{nonglyc_sites_count/total_sites_count*100:.2f}% (weight: {nonglyc_weight})")
print(f"Positive weight:\t{glyc_pos_weight}")

Total sites:448463
Glycosylated sites:	1607	0.36% (weight: 139.53422526446795)
Seen Non-glycosylated residues:	446856	99.64% (weight: 0.50179811840951)
Positive weight:	278.06845052893584


## Select proteins for validation/test partitions

Validation/Testing partitions have the following criteria:
 - Glycosylated Proteins must have at least one non-ambiguous site and one non-glycosylated site
 - Non-glycosylated Proteins must have at least two non-glycosylated site

In [11]:
candidate_trunc_certain_glyc_proteins = {protein_id for protein_id in trunc_certain_glyc_proteins if len(trunc_certain_glyc_proteins[protein_id]) >= 1}
candidate_trunc_glyc_proteins_nonglyc = {protein_id for protein_id in trunc_glyc_proteins_nonglyc if len(trunc_glyc_proteins_nonglyc[protein_id]) >= 1}
candidate_trunc_non_glyc_proteins = {protein_id for protein_id in trunc_non_glyc_proteins if len(trunc_non_glyc_proteins[protein_id]) >= 2}

candidate_trunc_proteins = (candidate_trunc_certain_glyc_proteins & candidate_trunc_glyc_proteins_nonglyc) | candidate_trunc_non_glyc_proteins

print(f"Validation/Test Partition Candidate Proteins: {len(candidate_trunc_proteins)}")

Validation/Test Partition Candidate Proteins: 363


## Run graph-part to create validation/test partitions from certain data

In [12]:
GRAPHPART_DIR.mkdir(exist_ok=True)
graphpart_input_string = f"'{GRAPHPART_INPUT_FILE}'"
graphpart_output_string = f"'{GRAPHPART_OUTPUT_FILE}'"


#length_sorted_proteins = sorted(proteins, key=lambda protein_id: len(proteins[protein_id].protein_seq), reverse=True)
length_sorted_candidate_trunc_proteins = sorted(candidate_trunc_proteins, key=lambda protein_id: len(proteins[protein_id].protein_seq), reverse=True)

with open(GRAPHPART_INPUT_FILE, 'w') as f:
    #for protein_id in length_sorted_proteins:
    for protein_id in length_sorted_candidate_trunc_proteins:
        sequence = proteins[protein_id].protein_seq
        seq_length = min(len(sequence), MAX_SEQ_LENGTH)
        #header = f"{protein_id}|priority={int(protein_id in candidate_trunc_proteins)}"
        header = f"{protein_id}|priority={int(protein_id in candidate_trunc_proteins)}"
        SeqIO.write(SeqRecord(Seq(sequence[:seq_length]), header, '', ''), f, "fasta")

In [13]:
def round_nearest(num: float, to: float) -> float:
    num, to = Decimal(str(num)), Decimal(str(to))
    return float(round(num / to) * to)

scaled_valtest_ratio = (len(proteins)*0.05)/len(candidate_trunc_proteins)
scaled_valtest_ratio_rounded = round_nearest(scaled_valtest_ratio, 0.05)
print(f"--val/test-ratio 0.05 scaled to candidate dataset: {scaled_valtest_ratio} and rounded to multiple of 0.05: {scaled_valtest_ratio_rounded}")

--val/test-ratio 0.05 scaled to candidate dataset: 0.10826446280991736 and rounded to multiple of 0.05: 0.1


In [14]:
display(Markdown("### Next cell will run following command (with quotes added):"))
!echo graphpart mmseqs2 --fasta-file {graphpart_input_string} --out-file {graphpart_output_string} --threshold {GRAPHPART_THRESHOLD} --val-ratio {scaled_valtest_ratio_rounded} --test-ratio {scaled_valtest_ratio_rounded}

### Next cell will run following command (with quotes added):

graphpart mmseqs2 --fasta-file /home/jakob/Cloudvault_new/Data/NetOGlyc5 data/GalNAc data/06-partitioning/netoglyc4_graphpart_partitions/graphpart_input.fasta --out-file /home/jakob/Cloudvault_new/Data/NetOGlyc5 data/GalNAc data/06-partitioning/netoglyc4_graphpart_partitions/graphpart_output.csv --threshold 0.3 --val-ratio 0.1 --test-ratio 0.1


In [15]:
!graphpart mmseqs2 --fasta-file {graphpart_input_string} --out-file {graphpart_output_string} --threshold {GRAPHPART_THRESHOLD} --val-ratio {scaled_valtest_ratio_rounded} --test-ratio {scaled_valtest_ratio_rounded}

Running in train-validation-test split mode with 10% test and 10% validation.
   lim  num  val
0   36  363    0
createdb --dbtype 1 /home/jakob/Cloudvault_new/Data/NetOGlyc5 data/GalNAc data/06-partitioning/netoglyc4_graphpart_partitions/graphpart_input.fasta temp/seq_db 

MMseqs Version:       	13.45111
Database type         	1
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[304] 0s 1ms
Time for merging to seq_db_h: 0h 0m 0s 2ms
Time for merging to seq_db: 0h 0m 0s 1ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 7ms
align temp/seq_db temp/seq_db temp/pref temp/align_db --alignment-mode 3 -e inf --seq-id-mode 1 --min-seq-id 0.3 

MMseqs Version:           	13.45111
Substitution matrix       	nucl:nucleotide.out,aa:blosum62.out
Add backtrace             	false
Alignment mode            	3
Alignment mode            	0
Allow wrapped scoring     	false


In [16]:
clusters = [set(), set(), set()]
with open(GRAPHPART_OUTPUT_FILE, 'r') as f:
    f_iter = iter(f)
    header = next(f_iter).strip().split(',')
    for line in f_iter:
        AC, priority, glycosylated, cluster = line.strip().split(',')
        priority = priority == "True"
        glycosylated = float(glycosylated) > 0
        cluster = int(float(cluster))
        clusters[cluster].add(AC)

In [17]:
[len(cluster) for cluster in clusters]

[291, 36, 36]

In [18]:
notebook_end_time = time.perf_counter()
print(f"Notebook took {notebook_end_time-notebook_start_time} seconds to run")

Notebook took 76.4992773180129 seconds to run
